In [1]:
import os
from dotenv import load_dotenv, find_dotenv
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

_ = load_dotenv(find_dotenv())

chat_model = ChatGroq(model = "llama-3.1-8b-instant")

## RunnablePassthrough
* It does not do anything to the input data.
* Let's see it in a very simple example: a chain with just RunnablePassthrough() will output the original input without any modification.

## RunnableLambda
* To use a custom function inside a LCEL chain we need to wrap it up with RunnableLambda.
* Let's define a very simple function to create Russian lastnames:

In [3]:
from langchain_core.runnables import RunnablePassthrough, RunnableLambda

def add_russ_last_name(name: str) -> str:
    return name + " ovich"

chain = RunnablePassthrough() | RunnableLambda(add_russ_last_name)

chain.invoke("Russ") 

'Russ ovich'

## RunnableParallel
* We will use RunnableParallel() for running tasks in parallel.
* This is probably the most important and most useful Runnable from LangChain.
* In the following chain, RunnableParallel is going to run these two tasks in parallel:
    * operation_a will use RunnablePassthrough.
    * operation_b will use RunnableLambda with the russian_lastname function.

In [5]:
from langchain_core.runnables import RunnableParallel

chain = RunnableParallel(
    {
        "op-1" : RunnablePassthrough(),
        "op-2" : RunnableLambda(add_russ_last_name)
    }
)

chain.invoke("Russ") 

{'op-1': 'Russ', 'op-2': 'Russ ovich'}

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("Tell me a joke about {subject}.")

chain = RunnableParallel(
    {
        "op-1" : RunnablePassthrough(),
        "subject" : RunnableLambda(add_russ_last_name),
        "op-3" : RunnablePassthrough()
    } 
) | prompt | chat_model | StrOutputParser()

chain.invoke("Russ")

'Why did Russ Owenich bring a ladder to the party? \n\nBecause he heard the drinks were on the house.'

# Advance use of RunnableParallel

In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

vectorstore = FAISS.from_texts(["inception bd focuses on providing content on Data Science, AI, ML, DL, CV, NLP, Python programming, etc. in English."], embedding=OpenAIEmbeddings())

template = """Answer the following question based on the context below.
Context: {context}
Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

retriever = vectorstore.as_retriever()

model = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

chain = (
    RunnableParallel({
        "context" : retriever ,
        "question" : RunnablePassthrough()
    }) | prompt | model | StrOutputParser()
)

response = chain.invoke("What topics does inception bd")

response

In [ ]:
from operator import itemgetter

template = """Answer the question based only on the following context:
{context}

Question: {question}

Answer in the following language: {language}
"""

prompt = ChatPromptTemplate.from_template(template)


chain = (
    {
        "context" : itemgetter("question") | retriever,
        "question" : itemgetter("question")
    } | prompt | model | StrOutputParser()
)

chain.invoke({"question": "What topics does inception bd?"})

# | Version                       | Use When                                          | Input Type                            | Strength                |
| ----------------------------- | ------------------------------------------------- | ------------------------------------- | ----------------------- |
| **RunnableParallel version**  | Simple RAG with a single input string             | `"question,text"`                     | Easy, clean             |
| **Dict + itemgetter version** | More complex pipelines with multiple input fields | `{"question": "...", "user_id": ...}` | More flexible, scalable |


# 🧠 Key Difference (short version)
RunnableParallel version

* Input: a raw string (single input )

* Both context + question derived from same input

* Simpler, less configurable

Dict routing version

* Input: a dict ( many input fields )

* You manually choose which key feeds which component

* Better for complex/production pipelines